<table style="width:100%; background-color: #EBF5FB">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Time series: Main Notebook</b>
      <ul>
        <li>Main Notebook</li>
        <li><a href="processing.ipynb">Processing Notebook</a></li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/time_series">Time series Data Package</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# 1. About Open Power System Data

This notebook is part of the project [Open Power System Data](http://open-power-system-data.org). Open Power System Data develops a platform for free and open data for electricity system modeling. We collect, check, process, document, and provide data that are publicly available but currently inconvenient to use. 
More info on Open Power System Data:
- [Information on the project on our website](http://open-power-system-data.org)
- [Data and metadata on our data platform](http://data.open-power-system-data.org)
- [Data processing scripts on our GitHub page](https://github.com/Open-Power-System-Data)

# 2. About Jupyter Notebooks and GitHub

This file is a [Jupyter Notebook](http://jupyter.org/). A Jupyter Notebook is a file that combines executable programming code with visualizations and comments in markdown format, allowing for an intuitive documentation of the code. We use Jupyter Notebooks for combined coding and documentation. We use Python 3 as programming language. All Notebooks are stored on [GitHub](https://github.com/), a platform for software development, and are publicly available. More information on our IT-concept can be found [here](http://open-power-system-data.org/it). See also our [step-by-step manual](http://open-power-system-data.org/step-by-step) how to use the dataplatform.

# 3. About this datapackage

We provide data in different chunks, or [data packages](http://frictionlessdata.io/data-packages/). The one you are looking at right now, [Time series](http://data.open-power-system-data.org/time_series/), contains various kinds of time series data in 15min, 30min or 60min resolution, namely:
- electricity consumption (load)
- wind and solar power: capacity, generation forecast, actual generation
- day-ahead spot prices

The main focus of this datapackage is German data, but we include data from other countries wherever possible.
The timeseries become available at different points in time depending on the sources. The full dataset is only available from 2015 onwards.
The data has been downloaded from the sources, resampled and merged in a large CSV file with hourly resolution. Additionally, the data available at a higher resolution (some renewables in-feed, 15 minutes) is provided in a separate file.

# 4. Data sources

The main data sources are the various European Transmission System Operators (TSOs) and the [ENTSO-E Data Portal](https://www.entsoe.eu/data/data-portal/Pages/default.aspx). Where no data is available from hte TSOs directly, data are taken from the [ENTSO-E Transparency Plstform](https://transparency.entsoe.eu). A complete list of data sources is provided on the [datapackage information website](http://data.open-power-system-data.org/time_series/). They are also contained in the JSON file that contains all metadata.

# 5. Naming conventions

{{import pandas as pd; pd.read_csv('input/notation.csv', index_col=list(range(4)))}}

# 6. License

This notebook as well as all other documents in this repository is published under the [MIT License](LICENSE.md).